In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import emoji

In [182]:
df = pd.read_csv("./data/output.csv")

In [183]:
# "review_text" nan are to be deleted. Other can stay.
df = df.dropna(subset=["review_text"])

# Remove invalid entries
df = df.loc[df["entry_date"] != "entry_date"]

# Remove duplicate entries
df = df.drop_duplicates(["offer_ref", "entry_id", "review_text"]).drop_duplicates("review_text")

# fix data types
df["entry_date"] = pd.to_datetime(df["entry_date"])
df["purchase_date"] = pd.to_datetime(df["purchase_date"])
df["entry_id"] = df["entry_id"].astype(int)
df["offer_ref"] = df["offer_ref"].astype(int)
df["score"] = df["score"].astype(float)

# Get Sentiment Cases based on score
df["sentiment"] = df["score"].apply(lambda x: "Positive" if x >= 4 else "Negative" if x <= 2 else "Neutral")

In [184]:
to_clean = df[["review_text", "sentiment"]].copy()

In [185]:
# Get rid of newlines
to_clean["review_text"] = to_clean["review_text"].replace("[\t\r\n\v\f\ufeff]", " ", regex=True)

In [186]:
# Remove whitespaces
to_clean["review_text"] = to_clean["review_text"].replace(" +", " ", regex=True)
to_clean[to_clean["review_text"].str.contains("  ")]

,review_text,sentiment


In [187]:
to_clean["has_emoji"] = to_clean["review_text"].transform(lambda x: np.any([emoji.is_emoji(c) for c in x]))
to_clean["all_emoji"] = to_clean["review_text"].transform(lambda x: np.all([emoji.is_emoji(c) for c in x]))

In [188]:
# 460 Reviews containing emojis
print(len(to_clean[to_clean["has_emoji"]]))
# 42 reviews containing only emojis
print(len(to_clean[to_clean["all_emoji"]]))

460
42


In [189]:
# You can sort of guess the sentiment of the review based on the emojis. But it's not consistent.
to_clean[to_clean["all_emoji"]].head(10)

,review_text,sentiment,has_emoji,all_emoji
0,😑😑,Negative,True,True
1091,😐,Negative,True,True
9417,👍,Neutral,True,True
10049,👍👍👍👍👍,Positive,True,True
10954,🤗,Positive,True,True
11175,🥳🎉,Positive,True,True
13724,👍🏻👍🏻,Negative,True,True
14707,👍👍,Neutral,True,True
24200,🙂,Positive,True,True
25320,😫,Negative,True,True


In [190]:
# 1927 reviews containing special unicode characters
to_clean[to_clean["review_text"].str.contains("[\t\r\n\v\f\ufeff]")]

,review_text,sentiment,has_emoji,all_emoji


In [196]:
to_clean["has_ok"] = to_clean["review_text"].str.lower().str.contains("\Wok\W")
to_clean[to_clean["has_ok"]] = to_clean[to_clean["has_ok"]].replace(r"\bok\b", "Ok", regex=True)

In [197]:
import spacy

In [198]:
nlp = spacy.load("pl_core_news_lg")

In [199]:
nlp("ok")

ok

In [201]:
to_clean[to_clean["has_ok"] & (to_clean["sentiment"] != "Positive")]["review_text"].iloc[2]

'Kupiłem za namową sąsiadów, którzy zachwycali się możliwością pieczenia 2 potraw jednocześnie. Dual Cooki nie są drogie, miałem Ok. 2000 zł na piekarnik (poprzedni zepsuł się po 13 latach, ale wiem, że takich urządzeń już nie ma...). Zdecydowałem się na ten model choć już w sklepie pojawiły się pierwsze wątpliwości... drzwiczki piekarnika trzaskają jak w maluchu, obudowa drzwi jest w pełni plastikowa i nie znalazłem żadnego uszczelnienie tej przegrody rozdzielającej piekarnik na 2 części. Sprzedawca stwierdził, że są to bzdety nie mające wpływu na jakość pieczenia i szczerze mówiąc przekonał mnie. Piekę głównie mięsiwa, a żona ciasta - piekarnik działa praktycznie codziennie, używamy go także do podgrzewania (nie lubię mikrofali). Rozczarowanie przyszło już po pierwszym pieczeniu - piekarnik bardzo długo się nagrzewa - temperaturę 220 stopni osiągnął po 20 minutach, mimo, że termostat pokazywał taką temperaturę już po 8-10 minutach - własny termometr pokazał, że piekarnik przekłamuje.

In [210]:
to_clean[to_clean.loc[:,"review_text"].str.contains("Ok\. [0-9]")]["review_text"].iloc[1]

'Więcej problemów niż korzyści. Po kilku miesiącach nie można było ustawić temperatury piekarnika. Różnica miedzy nastawieniem a rzeczywistą temperaturą wynosiła nawet 40 stopni C. Przyszedł Pan z serwisu, coś podłubał i przesłał do centrum serwisowego Samsunga, że jest Ok - nie było. Było gorzej. Ponowny kontakt z Centrum i oskarżenia z ich strony o konfabulacje z mojej strony. W końcu przyszli, wymienili cały moduł sterowania. Popracowało pół roku i ponownie analogiczne problemy. W zakresie temperatur od Ok. 150 do 200 stopni regulator nie działa. Aby uzyskać rzeczywistą temperaturę 180 stopni trzeba ustawić na regulatorze 200 lub więcej i pilnować, aby w odpowiednim momencie "skręcić" temperaturę. Ja mam pilnować? Nie po to wydałem niemałe pieniądze, abym teraz ja miał pilnować "ustawionej" temperatury.'